In [1]:
import h5py
import numpy as np
import pandas as pd
filepath1 =  'hdf5.test.1.h5'
filepath2 =  'hdf5.test.2.h5'
filepath3 =  'hdf5.test.3.h5'

In [7]:
# save ndarray to H5 file
a = np.array([
    [0,1,2],
    [2,3,1],
    [1,3,4]])
# write a H5 file
with h5py.File(filepath1, 'w') as f:
    # create a dataset
    f.create_dataset('test0', data=a)
# read a H5 file
with h5py.File(filepath1, 'r') as f:
    print("h5 file's keys: ", *f.keys())
    # get h5py.Dataset
    dset = f['test0']
    # get ndarray from h5py.Dataset
    ap = dset[()]
    print(ap)

h5 file's keys:  test0
[[0 1 2]
 [2 3 1]
 [1 3 4]]


In [3]:
# save pd.DataFrame to H5 file
d = {'patch_size': [1024, 1024, 512],
     'magnification': [10, 20, 40],
     'dataset_name': ['tn', 'ovr_mmrd', 'mmrd']}
df = pd.DataFrame(d)
with pd.HDFStore(filepath1) as s:
    s['test1'] = df

# must use h5py.File to read ndarray data
with h5py.File(filepath1, 'r') as f:
    print(f['test0'][()])

# must use pd.HDFStore to read DataFrame data
with pd.HDFStore(filepath1) as s:
    print(s['test1'])

[[0 1 2]
 [2 3 1]
 [1 3 4]]
   patch_size  magnification dataset_name
0        1024             10           tn
1        1024             20     ovr_mmrd
2         512             40         mmrd


In [10]:
# save ndarray to H5 file
a = np.array([
    [0,1,2],
    [2,3,1],
    [1,3,4]])
# write a H5 file
with h5py.File(filepath2, 'w') as f:
    try:
        f['gr1/gr2/test0']
    except Exception as e:
        print(e)
    try:
        gr.create_dataset('gr1/gr2/test0', data=a)
    except Exception as e:
        print(e)
    # create a group
    gr = f.create_group("gr1/gr2")
    # create a dataset
    gr.create_dataset('test0', data=a)
# read a H5 file
with h5py.File(filepath2, 'r') as f:
    print("h5 file's keys: ", *f.keys())
    print("gr1 in f?", 'gr1' in f)
    print("gr1/gr2 in f?", 'gr1/gr2' in f)
    print("gr1/gr2/test0 in f?", 'gr1/gr2/test0' in f)
    print("gr1/gr2/test0 is dataset?", isinstance(f['gr1/gr2/test0'], h5py.Dataset))
    print("gr1/gr2/test0 is group?", isinstance(f['gr1/gr2/test0'], h5py.Group))
    # get h5py.Dataset
    dset = f['gr1/gr2/test0']
    # get ndarray from h5py.Dataset
    ap = dset[()]
    print(ap)
    group = f['gr1/gr2']
    print("test0 in g?", 'test0' in group)
    print("test0 is dataset?", isinstance(group['test0'], h5py.Dataset))
    

'Unable to open object (component not found)'
Not a location id (invalid object ID)
h5 file's keys:  gr1
gr1 in f? True
gr1/gr2 in f? True
gr1/gr2/test0 in f? True
gr1/gr2/test0 is dataset? True
gr1/gr2/test0 is group? False
[[0 1 2]
 [2 3 1]
 [1 3 4]]
test0 in g? True
test0 is dataset? True


In [12]:
# save ndarray to H5 file
width = 4
height = 3
# write a H5 file
with h5py.File(filepath2, 'w') as f:
    f.create_dataset('test', (height, width,), dtype='i')
with h5py.File(filepath2, 'a') as f:
    dset = f['test']
    # write array horizontally
    dset[1,1:3] = np.array([-1,-1])
    # write array vertically
    dset[0:2,3] = np.array([1,1])
    # write number
    dset[0,1] = 2
    # write a single cell
    dset[0,2] = np.array([3])
with h5py.File(filepath2, 'r') as f:
    dset = f['test']
    print(dset[()])

[[ 0  2  3  1]
 [ 0 -1 -1  1]
 [ 0  0  0  0]]


In [19]:
a1 = np.array([
    [0,1],
    [2,3]])
a2 = np.array([
    [1,0],
    [0,1]])
a3 = np.array([
    [0,1],
    [1,0]])
a4 = np.array([
    [1,-1],
    [-1,1]])

with h5py.File(filepath3, 'w') as f:
    gr = f.create_group("gr11/gr21")
    dset = gr.create_dataset('a1', data=a1)
    # cannot create group gr11 since already exist
    # but can create group gr11/gr22 since diff. inner group
    try:
        gr = f.create_group('gr11')
    except ValueError as e:
        print(e)
    gr = f.create_group("gr11/gr22")
    gr.create_dataset('a2', data=a2)
    # can create dataset directly within groups
    dset = f.create_dataset('gr12/gr2/a3', data=a3)
    dset = f.create_dataset('gr13/gr2/gr3/a4', data=a4)

def get_h5_keys(f, key=''):
    keys = [ ]
    if isinstance(f, h5py.Dataset):
        return [key]
    else:
        for k, v in f.items():
            if key:
                tmp_key = f"{key}/{k}"
            else:
                tmp_key = k
            tmp_keys = get_h5_keys(v, key=tmp_key)
            keys.extend(tmp_keys)
        return keys
            
with h5py.File(filepath3, 'r') as f:
    for k in get_h5_keys(f):
        print(k)
    
# get list of H5 IDs
with h5py.File(filepath3, 'r') as f:
    for k1, v1 in f.items():
        print(k1)
        for k2, v2 in v1.items():
            print(' ', k2)
            for k3, v3 in v2.items():
                print('    ', k3)
                if isinstance(v3, h5py.Group):
                    for k4, v4 in v3.items():
                        print('      ', k4)
        
    

Unable to create group (name already exists)
gr11/gr21/a1
gr11/gr22/a2
gr12/gr2/a3
gr13/gr2/gr3/a4
gr11
  gr21
     a1
  gr22
     a2
gr12
  gr2
     a3
gr13
  gr2
     gr3
       a4
